In [6]:
# Combine_zips_with_any_intersection
import pandas as pd
import numpy as np
import datetime
import json
from haversine import haversine
import zipcodes
import googlemaps

In [2]:
zip_centers=json.load(open("/home/jian/Docs/Geo_mapping/center_of_rentrak_zip.json"))

In [3]:
data=pd.read_csv("/home/jian/Projects/Smoothie_King/TA/data_from_Jay/zip_within_3_mile.csv",dtype=str)

In [4]:
data['zip_list']=data['zip_within_3_mile'].apply(lambda x: list(set([str(y).zfill(5) for y in eval(x)])))
data['TA']=np.nan
data['TA']=1

df_TA_zips=pd.DataFrame({"store":[data['store_number'][0]]*len(data['zip_list'][0]),"zip":data['zip_list'][0],"TA":[1]*len(data['zip_list'][0])},index=[1]*len(data['zip_list'][0]))

In [5]:
data.head(2)

,store_number,store_name,address,city,state,zip_code,phone_number,store_hours,latitude,longitude,geo_accuracy,country,country_code,county,zip_within_3_mile,zip_list,TA
0,0,NaN,7215 Veterans Memorial Blvd. Ste. C,Metairie,LA,70003,(504) 457-4456,Monday 7:00 am 9:00 pm Tuesday 7:00 am 9:00 pm...,30.005821299999997,-90.22060079999999,SUBPREMISE: ROOFTOP,United States,US,Jefferson Parish,"['70060', '70006', '70065', 70003, '70062', '7...","[70065, 70063, 70062, 70060, 70003, 70006]",1
1,2,Smoothie King Veterans and Clearview,4539 Veterans Blvd.,Metairie,LA,70006,(504) 264-7880,Monday 6:30 am 10:00 pm Tuesday 6:30 am 10:00 ...,30.0058425,-90.1846858,PREMISE: ROOFTOP,United States,US,Jefferson Parish,"['70060', '70002', '70006', '70011', '70004', ...","[70009, 70183, 70005, 70004, 70011, 70002, 701...",1


In [5]:
TA_counter=1

for i in range(1,len(data)):
    intersection_zip=list(set(data['zip_list'][i]).intersection(set(df_TA_zips['zip'].unique().tolist())))
    if len(intersection_zip)==0:
        TA_counter+=1
        df_TA_zips=df_TA_zips.append(pd.DataFrame({"store":[data['store_number'][i]]*len(data['zip_list'][i]),"zip":data['zip_list'][i],"TA":[TA_counter]*len(data['zip_list'][i])},index=[i]*len(data['zip_list'][i]))).drop_duplicates()
        
    else:
        df_intersection=df_TA_zips[df_TA_zips['zip'].isin(intersection_zip)]
        group_df_intersection=df_intersection.groupby(['TA'])['zip'].count().to_frame().reset_index().sort_values(['zip'],ascending=False)
        selected_TA=group_df_intersection['TA'][0] 
        
        df_TA_zips_0=df_TA_zips[~df_TA_zips['TA'].isin(set(group_df_intersection['TA']))]
        df_TA_zips_1=df_TA_zips[df_TA_zips['TA'].isin(group_df_intersection['TA'].tolist())]
        df_TA_zips_1['TA']=selected_TA
        df_TA_zips=df_TA_zips_0.append(df_TA_zips_1).append(pd.DataFrame({"store":[data['store_number'][i]]*len(data['zip_list'][i]),"zip":data['zip_list'][i],"TA":[selected_TA]*len(data['zip_list'][i])},index=[i]*len(data['zip_list'][i]))).drop_duplicates()
        
dict_TA_zips=df_TA_zips.set_index('zip').to_dict()['TA']
dict_TA_store=df_TA_zips.set_index('store').to_dict()['TA']
data['TA']=data['store_number'].apply(lambda x: dict_TA_store[x])

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [6]:
df_ta_num_unique=data[['TA']].drop_duplicates().reset_index()
del df_ta_num_unique['index']
df_ta_num_unique['new_TA']=[x+1 for x in range(len(df_ta_num_unique))]

dict_ta_num_unique=df_ta_num_unique.set_index(['TA']).to_dict()['new_TA']
data['TA']=data['TA'].apply(lambda x: dict_ta_num_unique[x])
df_TA_zips['TA']=df_TA_zips['TA'].apply(lambda x: dict_ta_num_unique[x])


In [7]:

summary_store=data.groupby("TA")['store_number'].count().to_frame().reset_index().rename(columns={"store_number":"store_count"})
summary_zip=df_TA_zips[['TA','zip']].drop_duplicates().groupby("TA")['zip'].count().to_frame().reset_index().rename(columns={"zip":"zip_count"})
# summary_store_2=df_TA_zips[['TA','store']].drop_duplicates().groupby("TA")['store'].count().to_frame().reset_index().rename(columns={"store":"store_count_2"})
summary_store_list=data.groupby("TA")['store_number'].apply(list).to_frame().reset_index().rename(columns={"store_number":"store_list"})
summary_zip_list=df_TA_zips[['TA','zip']].drop_duplicates().groupby("TA")['zip'].apply(list).to_frame().reset_index().rename(columns={"zip":"zip_list"})



summary_by_TA=pd.merge(summary_store,summary_zip,on="TA",how="outer")
summary_by_TA=pd.merge(summary_by_TA,summary_store_list,on="TA",how="outer")
summary_by_TA=pd.merge(summary_by_TA,summary_zip_list,on="TA",how="outer")

# summary=pd.merge(summary,summary_store_2,on="TA",how="outer")
TA_Store_zip_list=data.groupby(['TA'])['zip_code'].apply(set).to_frame().reset_index().rename(columns={"zip_code":"store_zip_list"})
summary_by_TA=pd.merge(summary_by_TA,TA_Store_zip_list,on="TA",how="left")


In [8]:
summary_by_store_count=summary_by_TA.groupby(['store_count'])['TA'].count().to_frame().reset_index().rename(columns={"TA":"TA_count"})
summary_by_store_list=summary_by_TA.groupby(['store_count'])['TA'].apply(list).to_frame().reset_index().rename(columns={"TA":"TA_list"})
summary_by_store_count=pd.merge(summary_by_store_count,summary_by_store_list,on="store_count",how="outer")

In [9]:
df_zip_dist_in_TA=df_TA_zips[["TA","zip"]].drop_duplicates().reset_index()
del df_zip_dist_in_TA['index']

output_distance_zip_in_TA=pd.DataFrame()
counter_k=0
for ta,group in df_zip_dist_in_TA.groupby(['TA']):
    group=group.reset_index()
    del group['index']
    
    if len(group)>1:
    
        dist_list=[]

        for i in range(len(group)):
            zip_hold=group['zip'][i]

            if zip_hold not in zip_centers.keys():
                try:
                    zip_hold_center=(zipcodes.matching(zip_hold)[0]['lat'],zipcodes.matching(zip_hold)[0]['long'])
                except:
                    print("zip not found, ",zip_hold)

            else:
                zip_hold_center=zip_centers[zip_hold]

            for j in range(i+1,len(group)):
                zip_var=group['zip'][j]
                if zip_var not in zip_centers.keys():
                    try:
                        zip_var_center=(zipcodes.matching(zip_var)[0]['lat'],zipcodes.matching(zip_var)[0]['long'])
                    except:
                        print("zip not found, ",zip_hold)

                else:
                    zip_var_center=zip_centers[zip_var]

                try:
                    dist=haversine(zip_hold_center,zip_var_center,miles=True)
                except:
                    dist=np.nan

                dist_list=dist_list+[dist]
        df=pd.DataFrame({"TA":ta,"dist_min":min(dist_list),"dist_max":max(dist_list),"dist_median":np.median(dist_list),"All_dist":[dist_list]},index=[counter_k])
        counter_k+=1
        output_distance_zip_in_TA=output_distance_zip_in_TA.append(df)
    else:
        df=pd.DataFrame({"TA":ta,"dist_min":0,"dist_max":0,"dist_median":0,"All_dist":"single_zip"},index=[counter_k])
        output_distance_zip_in_TA=output_distance_zip_in_TA.append(df)
    

zip not found,  32758


In [10]:
zip_DMA=pd.read_excel("/home/jian/Docs/Geo_mapping/Zips by DMA by County16-17 nielsen.xlsx",dtype=str,skiprows=1)
zip_DMA=zip_DMA.iloc[:,[0,2]]
zip_DMA.columns=['zip','DMA']

In [11]:
df_city_state_in_TA=df_TA_zips[["TA","zip"]].drop_duplicates().reset_index()
del df_city_state_in_TA['index']

def city_of_zip(x):
    try:
        city=zipcodes.matching(x)[0]['city']
    except:
        city=np.nan
    return city

def state_of_zip(x):
    try:
        state=zipcodes.matching(x)[0]['state']
    except:
        state=np.nan
    return state
    
df_city_state_in_TA['city']=df_city_state_in_TA['zip'].apply(lambda x: city_of_zip(x))
df_city_state_in_TA['state']=df_city_state_in_TA['zip'].apply(lambda x: state_of_zip(x))
df_city_state_in_TA['city']=df_city_state_in_TA['city']+" ("+df_city_state_in_TA['state']+")"



In [12]:
df_DMA_in_TA=df_TA_zips[["TA","zip"]].drop_duplicates().reset_index()
del df_DMA_in_TA['index']
df_DMA_in_TA=pd.merge(df_DMA_in_TA,zip_DMA,on="zip",how="left")
df_DMA_in_TA.head(2)

,TA,zip,DMA
0,3,70031,NEW ORLEANS
1,3,70087,NEW ORLEANS


In [13]:
df_city_TA_list=df_city_state_in_TA.groupby(['TA'])['city'].apply(set).to_frame().reset_index()
df_city_TA_list=df_city_TA_list.rename(columns={"city":"city_list"})
df_state_TA_list=df_city_state_in_TA.groupby(['TA'])['state'].apply(set).to_frame().reset_index()
df_state_TA_list=df_state_TA_list.rename(columns={"state":"state_list"})
df_DMA_TA_list=df_DMA_in_TA.groupby(['TA'])['DMA'].apply(set).to_frame().reset_index()
df_DMA_TA_list=df_DMA_TA_list.rename(columns={"DMA":"DMA_list"})


In [14]:
df_city_state_in_TA.head(2)

,TA,zip,city,state
0,3,70031,AMA (LA),LA
1,3,70087,SAINT ROSE (LA),LA


In [15]:
counter_k
df_primary_city_state=pd.DataFrame()
df_primary_DMA=pd.DataFrame()
for ta,group in df_city_state_in_TA.groupby(['TA']):
    df_city=group.groupby(['city'])['zip'].count().to_frame().reset_index().sort_values(['zip'],ascending=False).reset_index()
    del df_city['index']
    primary_city=df_city['city'][0]
    
    df_state=group.groupby(['state'])['zip'].count().to_frame().reset_index().sort_values(['zip'],ascending=False).reset_index()
    del df_state['index']
    primary_state=df_state['state'][0]
    
    df=pd.DataFrame({"TA":ta,"Primary_City":primary_city,"Primary_State":primary_state},index=[counter_k])
    counter_k+=1
    df_primary_city_state=df_primary_city_state.append(df)

    
for ta,group in df_DMA_in_TA.groupby(['TA']):
    df_DMA=group.groupby(['DMA'])['zip'].count().to_frame().reset_index().sort_values(['zip'],ascending=False).reset_index()
    del df_DMA['index']
    primary_DMA=df_DMA['DMA'][0]
    
    df=pd.DataFrame({"TA":ta,"Primary_DMA":primary_DMA},index=[counter_k])
    counter_k+=1
    df_primary_DMA=df_primary_DMA.append(df)

In [16]:
summary_by_TA=pd.merge(summary_by_TA,df_city_TA_list,on="TA",how="left")
summary_by_TA=pd.merge(summary_by_TA,df_state_TA_list,on="TA",how="left")
summary_by_TA=pd.merge(summary_by_TA,df_DMA_TA_list,on="TA",how="left")

summary_by_TA=pd.merge(summary_by_TA,df_primary_city_state,on="TA",how="left")
summary_by_TA=pd.merge(summary_by_TA,df_primary_DMA,on="TA",how="left")

summary_by_TA=pd.merge(summary_by_TA,output_distance_zip_in_TA,on="TA",how="left")

In [17]:
writer=pd.ExcelWriter("/home/jian/Projects/Smoothie_King/TA/SmoothieKing_TA_3_miles_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
data.to_excel(writer,"output_TA",index=False)
df_TA_zips.to_excel(writer,"zip_TA",index=False)
summary_by_TA.to_excel(writer,"summary_by_TA",index=False)
summary_by_store_count.to_excel(writer,"summary_by_store_count",index=False)
writer.save()